In [ ]:
import sys, argparse, logging
import pymssql
import _mssql
import datetime
from decimal import Decimal
import pymongo
from pymongo import MongoClient


from tmqr.settings import *

try:
    from tmqr.settings_local import *
except:
    pass



# SQL Server credentials
SQL_HOST = 'h9ggwlagd1.database.windows.net'
SQL_USER = 'modelread'
SQL_PASS = '4fSHRXwd4u'

In [ ]:
sql_conn = pymssql.connect(SQL_HOST, SQL_USER + "@" + SQL_HOST, SQL_PASS, 'TMLDB')

In [ ]:
def convert_dates(values):
    k,v = values
    if type(v) == datetime.date:
        return k, datetime.datetime.combine(
                v,
                datetime.datetime.min.time())
    if type(v) == Decimal:
        return k, float(v)
    else:
        return k, v

In [ ]:
instruments = {}
colname = 'instruments'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    instruments[data['idinstrument']] = data

In [ ]:
instruments

In [ ]:
contracts = {}
colname = 'contracts'
#qry = 'SELECT TOP 10 * FROM cqgdb.tbl{0}'.format(colname)
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    tokens = data['contractname'].split('.')
    if '.' not in data['contractname'] or len(tokens) != 3:
        print('Wrong contract name: ' + data['contractname'])
        continue
    ctype, cmkt, cname = tokens
    
    idinstrument = data['idinstrument']
    
    if idinstrument not in instruments:
        print("idinstrument = {0} is not found for {1}".format(idinstrument,data['contractname']))
        continue
    
    underlying = instruments[idinstrument]['exchangesymbol']
    expiration = datetime.datetime.strptime(data['expirationdate'], '%Y-%m-%d')
    contract = '{0}.{1}{2}'.format(underlying, 
                                  data['month'],
                                  str(int(data['year']))[2:])
    ticker = '{0}.{1}.{2}.{3}'.format(cmkt, 
                                      ctype, 
                                      contract,
                                      expiration.strftime('%y%m%d')       
                                    )
    contracts[data['idcontract']] = {'tckr': ticker, 
                                     'contr': contract, 
                                     'type': ctype, 
                                     'underlying': '{0}.{1}'.format(cmkt, underlying),
                                     'instr': '{0}.{1}'.format(cmkt, underlying),
                                     'exp': expiration,
                                     'mkt': cmkt,
                                     'extra_data': {
                                            'month': data['month'],
                                            'monthint': data['monthint'],
                                            'year': data['year'],
                                            'name': data['contractname'],
                                            'sqlid': data['idcontract'],
                                        }
                                     }
    #print(ticker)
    #break

In [ ]:
# Init mongo asset index
client = MongoClient(MONGO_CONNSTR)
mongo_db = client[MONGO_DB]

In [ ]:
# Storing futures
mongo_collection = mongo_db['asset_index']

mongo_collection.create_index([('tckr', pymongo.ASCENDING)], unique=True)

mongo_collection.create_index([('contr', pymongo.ASCENDING), 
                               ('mkt', pymongo.ASCENDING), 
                               ('type', pymongo.ASCENDING)])

cnt = 0
dup_cnt = 0
for c in contracts.values():
    try:
        mongo_collection.insert_one(c)
        #mongo_collection.replace_one({'tckr': c['tckr']}, c, upsert=True)
        cnt += 1
    except pymongo.errors.DuplicateKeyError:
        #print("Duplicated record: " + c['tckr'])
        dup_cnt += 1
        
print("Records added: {0} Duplicated: {1}".format(cnt, dup_cnt))       
    

In [ ]:
colname = 'options'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
cnt = 0
dup_cnt = 0
for row in c2:
    data = dict(map(convert_dates, row.items()))
    #print(data)
    ctype, cmkt, cname = data['optionname'].split('.')
    
    if data['idcontract'] not in contracts:
        continue
    
    underlying_dict = contracts[data['idcontract']]
    underlying = underlying_dict['tckr'].replace('US.', '').replace('.','-') #"{0}-{1}".format(underlying_dict['type'], underlying_dict['contr'])
    expiration = datetime.datetime.strptime(data['expirationdate'], '%Y-%m-%d')
    
    ticker = '{0}.{1}.{2}.{5}@{6}'.format(cmkt, 
                                     ctype, 
                                     underlying, 
                                     data['optionmonth'],
                                     str(int(data['optionyear']))[2:],
                                     expiration.strftime('%y%m%d'),
                                     data['strikeprice']   
                                    )
    opt_record = {'tckr': ticker, 
                 'type': ctype, 
                 'underlying': underlying_dict['tckr'],
                 'exp': expiration,
                 'instr': underlying_dict['instr'],
                 'mkt': cmkt,
                 'opttype': data['callorput'],
                 'strike': data['strikeprice'],
                 'optcode': data['optioncode'],
                 'extra_data': {
                        'month': data['optionmonth'],
                        'monthint': data['optionmonthint'],
                        'year': data['optionyear'],
                        'name': data['optionname'],
                        'sqlid': data['idoption'],
                    }
                 }
    #print(ticker)
    #break
    try:        
        mongo_collection.insert_one(opt_record)
        cnt += 1
    except pymongo.errors.DuplicateKeyError:
        dup_cnt += 1
        #print("Duplicated record: " + c['tckr'])

print("Records added: {0} Duplicated: {1}".format(cnt, dup_cnt))

In [ ]:
contracts

In [ ]:
opt_record

In [ ]:
exchanges = {}
colname = 'exchange'
qry = 'SELECT * FROM cqgdb.tbl{0}'.format(colname)
logging.debug(qry)
c2 = sql_conn.cursor(as_dict=True)
c2.execute(qry)
for row in c2:
    exchanges[float(row['idexchange'])] = row
    print(row)

In [ ]:
EXCHANGE_NAMESPACE = 'US'

for iid, instr in instruments.items():
    #print(instr)
    res = {
        'tckr': "{0}.{1}".format(EXCHANGE_NAMESPACE, instr['exchangesymbol']),
        'ticksize': instr['ticksize'],
        'tickvalue': instr['tickvalue'],
        'mkt': EXCHANGE_NAMESPACE,
        'exchange': exchanges[instr['idexchange']]['exchange'],
        'description': instr['description'],
        'exchangesymbol': instr['exchangesymbol'],
        'extra_data': {          
        }
        
    }
    res['extra_data'].update(exchanges[instr['idexchange']])
    if res['exchangesymbol'] == 'ES':
        print(res)
        break
    